In [23]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.util import ngrams


In [2]:
print (pd.__name__, pd.__version__)

pandas 0.22.0


In [11]:
cd data

/Users/jasmindial/Desktop/gender_in_movies/data


In [33]:
df = pd.read_pickle("movies_text.p")

In [15]:
df.head()

,gender_to,movie_id,gender_from,char_id_from,char_id_to,line_id,text,movie_year,genre
0,0,m0,1,u0,u2,L194,Can we make this quick? Roxanne Korrine and A...,1999,comedy
1,0,m0,1,u0,u2,L195,"Well, I thought we'd start with pronunciation,...",1999,comedy
2,0,m0,1,u0,u2,L196,Not the hacking and gagging and spitting part....,1999,comedy
3,0,m0,1,u0,u2,L197,Okay... then how 'bout we try out some French ...,1999,comedy
4,0,m0,1,u0,u2,L198,You're asking me out. That's so cute. What's ...,1999,comedy


In [34]:
test = df[:10]

In [47]:
grams_df = pd.DataFrame({'grams': [], 'line_id': []})

for index, row in test.iterrows():
    
    #create list of bigrams for each text
    text = row['text']    
    token = nltk.word_tokenize(text)
    bigrams = ngrams(token, 2)
    grams_list = [gram for gram in bigrams]
    
    #create dict of bigrams
    line = row['line_id']
    gen = row['gender_from']
    d = {'grams': grams_list, 'line_id': np.repeat(line, len(grams_list)), 'gender_from': np.repeat(gen, len(grams_list))}
    grams_df = pd.concat([grams_df, pd.DataFrame(d)], axis = 0)


L194
    gender_from                     grams line_id
0           1.0                 (Can, we)    L194
1           1.0                (we, make)    L194
2           1.0              (make, this)    L194
3           1.0             (this, quick)    L194
4           1.0                (quick, ?)    L194
5           1.0              (?, Roxanne)    L194
6           1.0        (Roxanne, Korrine)    L194
7           1.0            (Korrine, and)    L194
8           1.0             (and, Andrew)    L194
9           1.0         (Andrew, Barrett)    L194
10          1.0            (Barrett, are)    L194
11          1.0             (are, having)    L194
12          1.0              (having, an)    L194
13          1.0          (an, incredibly)    L194
14          1.0  (incredibly, horrendous)    L194
15          1.0      (horrendous, public)    L194
16          1.0          (public, break-)    L194
17          1.0              (break-, up)    L194
18          1.0                  (up, on)    

In [48]:
grams_df

,gender_from,grams,line_id
0,1.0,"(Can, we)",L194
1,1.0,"(we, make)",L194
2,1.0,"(make, this)",L194
3,1.0,"(this, quick)",L194
4,1.0,"(quick, ?)",L194
5,1.0,"(?, Roxanne)",L194
6,1.0,"(Roxanne, Korrine)",L194
7,1.0,"(Korrine, and)",L194
8,1.0,"(and, Andrew)",L194
9,1.0,"(Andrew, Barrett)",L194
